In [ ]:
# to read files from google drive, in case you're running on colab
'''from google.colab import drive
drive.mount('/content/drive', force_remount=True)'''
data_dir = 'path to the project directory'
# using the following repo to detect faces
!pip3 install face_recognition

##  Import Datasets

Make sure that you've downloaded the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dog_images`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home directory, at location `/lfw`.  

In the code cell below, we save the file paths for both the human (LFW) dataset and dog dataset in the numpy arrays `human_files` and `dog_files`.

In [ ]:
# download the datasets and extract the zip files
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip
!unzip dogImages.zip -d dog_images
!mv dog_images/dogImages/* dog_images/
!rm -r dog_images/dogImages
!unzip lfw.zip
!rm -f dogImages.zip lfw.zip
!rm -r __MACOSX

In [ ]:
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("lfw/*/*"))
dog_files = np.array(glob("dog_images/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

In [ ]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

import face_recognition
import os
import torch
from torchvision import datasets, transforms

### Write a Human Face Detector

We can use this procedure to write a function that returns `True` if a human face is detected in an image and `False` otherwise.  This function, aptly named `face_detector`, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path.
def face_detector(img_path):
    image = face_recognition.load_image_file(img_path)
    face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")
    return len(face_locations) > 0

In [ ]:
from tqdm import tqdm

human_files_short = human_files[:100]
dog_files_short = dog_files[:100]

## Test the performance of the face_detector algorithm 
## on the images in human_files_short and dog_files_short.
humans_detected_1 = 0
for each_link in human_files_short:
    if face_detector(each_link):
        humans_detected_1 += 1
        
humans_detected_2 = 0
for each_link in dog_files_short:
    if face_detector(each_link):
        humans_detected_2 += 1
        
print(humans_detected_1, humans_detected_2)

### (IMPLEMENTATION) Making Predictions with a Pre-trained Model

In the next code cell, you will write a function that accepts a path to an image (such as `'dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg'`) as input and returns the index corresponding to the ImageNet class that is predicted by the pre-trained resnet152 model.  The output should always be an integer between 0 and 999, inclusive.

Before writing the function, make sure that you take the time to learn  how to appropriately pre-process tensors for pre-trained models in the [PyTorch documentation](http://pytorch.org/docs/stable/torchvision/models.html).

In [ ]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
batch_size = 64

train_transforms = transforms.Compose([transforms.Resize(300),
                                transforms.RandomRotation(5),
                                transforms.RandomCrop(224),
                                transforms.RandomHorizontalFlip(0.1),
                                transforms.RandomVerticalFlip(0.1),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder('dog_images/train', transform=train_transforms)
valid_dataset = datasets.ImageFolder('dog_images/valid', transform=valid_transforms)
test_dataset = datasets.ImageFolder('dog_images/test', transform=valid_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size)

loaders = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

### (IMPLEMENTATION) Write a Dog Detector

While looking at the [dictionary](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a), you will notice that the categories corresponding to dogs appear in an uninterrupted sequence and correspond to dictionary keys 151-268, inclusive, to include all categories from `'Chihuahua'` to `'Mexican hairless'`.  Thus, in order to check to see if an image is predicted to contain a dog by the pre-trained VGG-16 model, we need only check if the pre-trained model predicts an index between 151 and 268 (inclusive).

Use these ideas to complete the `dog_detector` function below, which returns `True` if a dog is detected in an image (and `False` if not).

### (IMPLEMENTATION) Assess the Dog Detector

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, usingSGD=True):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            outputs = model(data)
            
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()
                ## update the average validation loss
                outputs = model(data)
                
                loss = criterion(outputs, target)
                valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
         
        if(usingSGD):
            lr_scheduler.step(valid_loss_min)
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if(valid_loss < valid_loss_min):
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model

In [ ]:
# use transfer learning to train vgg16 for higher accuracy at detecting dogs
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict

vgg16_transfer = models.vgg16(pretrained=True)
for param in vgg16_transfer.parameters():
    param.requires_grad = False
    
classifier_name, old_classifier = vgg16_transfer._modules.popitem()
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(old_classifier[0].in_features, 4096)),
                                        ('relu1', nn.ReLU()),
                                        ('drop1', nn.Dropout(0.7)),
                                        ('fc2', nn.Linear(4096, 4096)),
                                        ('relu2', nn.ReLU()),
                                        ('drop2', nn.Dropout(0.5)),
                                        ('output', nn.Linear(4096, 1))
                                           ]))
vgg16_transfer.add_module(classifier_name, classifier)
#print(vgg16_transfer)
use_cuda = torch.cuda.is_available()
if use_cuda:
    vgg16_transfer = vgg16_transfer.cuda()

In [ ]:
# specifying loss function and optimizer
loss_dog = lambda output, _ : torch.mean((output-1)**2) 
optimizer_dog = optim.Adam(vgg16_transfer.parameters(), lr=0.000001)

# train the model
vgg16_transfer = train(10, loaders, vgg16_transfer, optimizer_dog, loss_dog, use_cuda, 'model_dog.pt', False)

In [ ]:
# load the model that got the best validation accuracy
vgg16_transfer.load_state_dict(torch.load(data_dir+'saved_models/model_dog.pt'))

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def VGG16_predict(img_path):
    '''   
    Args:
        img_path: path to an image
        
    Returns:
        Index corresponding to resnet152 model's prediction
    '''
    
    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image
    image = Image.open(img_path)
    t1 = transforms.RandomResizedCrop(224)
    t2 = transforms.ToTensor()
    t3 = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    image = t1(image)
    image = t2(image)
    image = t3(image)
    image = image.unsqueeze(0)
    if use_cuda:
        image = image.cuda()
    #print(image.shape)
    vgg16_transfer.eval()
    output = vgg16_transfer(image)
        
    #print(output)
    return output

In [ ]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    output = VGG16_predict(img_path)
    #print(output)
    return output > 0.5 # true/false

In [ ]:
### Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.
dogs_detected = 0
for pic in human_files_short:
    if dog_detector(pic):
        dogs_detected += 1
print(dogs_detected, 'dogs detected in', len(human_files_short), 'human images')

dogs_detected = 0
for pic in dog_files_short:
    if dog_detector(pic):
        dogs_detected += 1
print(dogs_detected, 'dogs detected in', len(dog_files_short), 'dog images')

## Create a CNN to Classify Dog Breeds (using Transfer Learning)

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
model_transfer = models.resnet152(pretrained=True)
# uncomment the following to freeze the pre-trained layers
'''ct = 0
layer_to_freeze = 7
for child in model_transfer.children():
    ct += 1
    if ct < layer_to_freeze:
        for param in child.parameters():
            param.requires_grad = False'''
    
classifier_name, old_classifier = model_transfer._modules.popitem()
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(old_classifier.in_features, 133))]))
model_transfer.add_module(classifier_name, classifier)
if use_cuda:
    model_transfer = model_transfer.cuda()
#model_transfer

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.fc.parameters(), 0.09, momentum = 0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_transfer, mode='min', factor=0.1, patience=2)
n_epochs = 50

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
model_transfer = train(n_epochs, loaders, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

In [ ]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load(data_dir+'saved_models/model_transfer_89.pt'))

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loaders['test']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### A function that takes a path to an image as input
### and returns the dog breed that is psredicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in train_dataset.classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    global model_transfer
    image = Image.open(img_path)
    t1 = transforms.Resize(256)
    t2 = transforms.CenterCrop(224)
    t3 = transforms.ToTensor()
    t4 = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    image = t4(t3(t2(t1(image))))
    image = image.unsqueeze(0)
    if(use_cuda):
        image = image.cuda()
        model_transfer = model_transfer.cuda()
    #print('lol what\'s happening')
    model_transfer.eval()
    output_list = model_transfer(image).tolist()[0]
    #print(output_list)
    max_value = max(output_list)
        
    #print(output)
    return class_names[output_list.index(max_value)]

---
<a id='step5'></a>
## The Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
def run_app(img_path):
    ## handle cases for a human face, dog, and neither
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    
    if dog_detector(img_path) and not face_detector(img_path):
        print("You have been visited by the good luck bringing doggo, your lucky breed is...\n", predict_breed_transfer(img_path))
        
    elif face_detector(img_path):
        print("Aaaaand your spirit animal is...\n", predict_breed_transfer(img_path))
    else:
        print("bruh! with right data comes right output, without, error messages")


### (IMPLEMENTATION) Test the Algorithm on Sample Images!

In [ ]:
for file in np.hstack((human_files[69:72], dog_files[69:72])):
    run_app(file)

In [ ]:
#testing the algorithm on images from my pc
local_images = os.listdir(data_dir+'local_images')
for each_image in local_images:
    if not each_image.startswith('.'):
        run_app(data_dir+'local_images/'+each_image)